In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.co

In [2]:
# Import packages
from pyspark.sql import SparkSession
from pyspark import SparkFiles
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# Read in the dataset from the AWS S3 bucket.
url = "https://2u-data-curriculum-team.s3.amazonaws.com/nflx-data-science-adv/week-5/Assignment/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sale=spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
home_sale.show()

+---+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|_c0|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+---+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|  0|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|  1|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|  2|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|  3|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|  4|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|  5|5aa00529-0533-46b...|2019-01-30|      2017|

In [4]:
# 1. Create a temporary view of the DataFrame.
home_sale.createOrReplaceTempView("home_sales")

In [6]:
# 2. What is the average price for a four bedroom house sold in 2019 rounded to two decimal places?
sql = """
SELECT
  ROUND(AVG(price),2) as AVERAGE_PRICE
 FROM home_sales
 WHERE date like "2019%" AND bedrooms=4 
"""
spark.sql(sql).show()

+-------------+
|AVERAGE_PRICE|
+-------------+
|     300263.7|
+-------------+



In [7]:
# 3. What is the average price of a home for each year that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
sql = """
SELECT
    YEAR(date) as YEAR, ROUND(AVG(price),2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY YEAR
ORDER BY YEAR DESC
"""
spark.sql(sql).show()

+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    314421.54|
|2021|    313218.09|
|2020|    315949.81|
|2019|    315782.18|
+----+-------------+



In [8]:
# 4. What is the average price of a home for each year that have 3 bedrooms, 3 bathrooms, and are less than 2,000 square feet rounded to two decimal places?
sql = """
SELECT
YEAR(date) AS YEAR, ROUND(AVG(price),2) AS 
FROM home_sales
WHERE (
  bedrooms=3
  AND bathrooms=3
  AND sqft_living<2000
)
GROUP BY YEAR
ORDER BY YEAR DESC
"""
spark.sql(sql).show()


+----+---------+
|YEAR|       AS|
+----+---------+
|2022|283031.02|
|2021| 282452.7|
|2020|291973.53|
|2019|280167.38|
+----+---------+



In [9]:
# 5. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than $350,000?
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()


sql = """
  SELECT
    view, ROUND(avg(price),2) AS AVERAGE_PRICE
  FROM home_sales
  WHERE price>350000 
  GROUP BY view
  ORDER BY view DESC

"""
spark.sql(sql).show()
print("--- %s seconds ---" % (time.time() - start_time)) 



+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.7823269367218018 seconds ---


In [ ]:
# 6. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [ ]:
# 7. Check if the table is cached.
spark.catalog.isCached("home_sales")

True

In [ ]:
# 8. Using the cached data, run the query that filters out the view ratings with average price of less than $350,000. 
# Determine the runtime and compare it to uncached runtime.

start_time1=time.time()

sql = """
  SELECT
  view, ROUND(avg(price),2) AS AVERAGE_PRICE
  FROM home_sales
  WHERE price>350000 
  GROUP BY view
  ORDER BY view DESC
"""
spark.sql(sql).show()

print("--- %s seconds ---" % (time.time() - start_time1)) 

print("The time difference for Uncahed Data Over Cached data is: ",(time.time() - start_time)-(time.time() - start_time1))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.3512241840362549 seconds ---
The time difference for Uncahed Data Over Cached data is:  174.02287435531616


In [13]:
# 9. Partition the home sales dataset by the date_built field.
home_sale.write.partitionBy("date_built").mode("overwrite").parquet("home_sales")

In [14]:
# 10.Read the formatted parquet data.
df=spark.read.parquet("home_sales")


In [15]:
# 11. Create a temporary table for the parquet data.
df.createOrReplaceTempView("sales")

In [16]:
df.show(10)

+---+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|_c0|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+---+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|  9|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
| 12|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
| 17|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
| 18|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
| 28|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
| 29|a18515a2-86f3-46b...|2022-02-18|419543|    

In [37]:
# 12. Run the query that filters out the view ratings with average price of less than $350,000 with the parquet DataFrame. 
# Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 


start_time=time.time()
sql = """
SELECT view, ROUND(AVG(price),2) as AVERAGE
FROM sales
WHERE price<350000
GROUP BY view
ORDER BY view DESC
"""
spark.sql(sql).show()
print("--- %s seconds ---" % (time.time() - start_time)) 



+----+---------+
|view|  AVERAGE|
+----+---------+
|  50|237517.97|
|  49|238078.42|
|  48|234616.34|
|  47|240530.23|
|  46|238128.72|
|  45|232313.97|
|  44|233404.96|
|  43| 237204.1|
|  42|235743.47|
|  41|235149.62|
|  40|238056.82|
|  39|232603.52|
|  38|239252.42|
|  37| 236525.8|
|  36|227436.03|
|  35|231676.29|
|  34|232111.68|
|  33|237419.87|
|  32|238639.35|
|  31|238947.13|
+----+---------+
only showing top 20 rows

--- 0.37482619285583496 seconds ---


In [39]:
# 13. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [40]:
# 14. Check if the home_sales is no longer cached
if spark.catalog.isCached("home_sales"):
  print("a table is till cached")
else:
  print("all clear")

all clear
